In [7]:
import os
import pandas as pd
import numpy as np
import sys
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
import sklearn.metrics as metrics
from sklearn.metrics import brier_score_loss, roc_curve, auc, confusion_matrix, roc_auc_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse
import matplotlib.pyplot as plt
from mizani.formatters import percent_format

import warnings
warnings.filterwarnings('ignore')


In [222]:
data = pd.read_csv("https://raw.githubusercontent.com/Jk33033/Data_Analysis3/main/assignment2/listings.csv")

In [9]:
data.describe()

,id,scrape_id,description,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,1.360100e+04,1.360100e+04,0.0,1.360100e+04,13601.000000,13601.000000,13601.000000,13601.000000,13601.000000,0.0,...,12107.000000,12107.000000,12107.000000,12106.000000,12106.000000,13601.000000,13601.000000,13601.000000,13601.000000,12088.000000
mean,3.217099e+17,2.023122e+13,NaN,2.053080e+08,35.036836,53.413867,41.153024,-8.613542,3.644732,NaN,...,4.727194,4.799739,4.794398,4.727122,4.634227,19.767885,17.733770,1.819425,0.162267,1.667862
std,4.049059e+17,0.000000e+00,NaN,1.803547e+08,111.773227,222.746900,0.067840,0.059958,2.124080,NaN,...,0.371016,0.321503,0.338532,0.327357,0.388097,43.891032,41.638831,4.623776,1.376186,1.604635
min,4.133900e+04,2.023122e+13,NaN,1.442280e+05,1.000000,1.000000,40.767760,-8.784090,1.000000,NaN,...,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.010000
25%,2.451509e+07,2.023122e+13,NaN,3.903119e+07,2.000000,2.000000,41.144040,-8.620660,2.000000,NaN,...,4.630000,4.750000,4.740000,4.620000,4.530000,2.000000,1.000000,0.000000,0.000000,0.400000
50%,4.606103e+07,2.023122e+13,NaN,1.437019e+08,5.000000,6.000000,41.149360,-8.611870,3.000000,NaN,...,4.830000,4.890000,4.890000,4.810000,4.720000,5.000000,3.000000,0.000000,0.000000,1.160000
75%,7.344801e+17,2.023122e+13,NaN,3.785917e+08,18.000000,21.000000,41.157417,-8.604530,4.000000,NaN,...,4.960000,4.980000,4.990000,4.940000,4.850000,14.000000,11.000000,1.000000,0.000000,2.500000
max,1.047130e+18,2.023122e+13,NaN,5.511028e+08,2457.000000,5521.000000,41.459634,-8.152370,16.000000,NaN,...,5.000000,5.000000,5.000000,5.000000,5.000000,260.000000,242.000000,37.000000,23.000000,14.790000


In [10]:
data.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,41339,https://www.airbnb.com/rooms/41339,20231217045113,2023-12-17,city scrape,Rental unit in Porto · ★4.73 · 1 bedroom · 3 b...,NaN,O apartamento fica perto da Fundação de Serral...,https://a0.muscache.com/pictures/miso/Hosting-...,180050,...,4.82,4.55,4.64,29049/AL,f,1,1,0,0,0.17
1,55111,https://www.airbnb.com/rooms/55111,20231217045113,2023-12-17,previous scrape,Rental unit in Paredes Municipality · ★4.79 · ...,NaN,NaN,https://a0.muscache.com/pictures/b3c38006-5fdf...,259711,...,4.89,4.78,4.78,7563/AL,f,3,3,0,0,0.19
2,73828,https://www.airbnb.com/rooms/73828,20231217045113,2023-12-17,previous scrape,Rental unit in Estação · ★4.86 · 2 bedrooms · ...,NaN,NaN,https://a0.muscache.com/pictures/ffb19299-425c...,259711,...,5.00,4.93,4.79,7563/AL,f,3,3,0,0,0.13
3,86388,https://www.airbnb.com/rooms/86388,20231217045113,2023-12-17,city scrape,Rental unit in Porto · ★4.75 · 1 bedroom · 1 b...,NaN,NaN,https://a0.muscache.com/pictures/850099/b80470...,185209,...,4.93,4.89,4.74,6995/AL,f,8,2,6,0,1.63
4,87873,https://www.airbnb.com/rooms/87873,20231217045113,2023-12-17,city scrape,Rental unit in Porto · ★4.78 · 2 bedrooms · 2 ...,NaN,OPorto Apartment Triplex is located on Rua San...,https://a0.muscache.com/pictures/33686513/c54f...,480980,...,4.94,4.50,4.80,634/AL,f,4,2,2,0,1.69


In [11]:
data.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [12]:
data.neighbourhood_cleansed.value_counts()

neighbourhood_cleansed
Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, Vitória    6987
Bonfim                                                  1391
Santa Marinha e São Pedro da Afurada                     643
Lordelo do Ouro e Massarelos                             538
Paranhos                                                 526
                                                        ... 
Cristelo                                                   1
Negrelos (São Tomé)                                        1
Sanguedo                                                   1
Fornos                                                     1
Gandra                                                     1
Name: count, Length: 152, dtype: int64

# EDA

In [217]:
data["accommodates"].info

<bound method Series.info of 0        5.0
1        5.0
2        5.0
3        2.0
4        4.0
        ... 
13596    4.0
13597    3.0
13598    3.0
13599    6.0
13600    3.0
Name: accommodates, Length: 13601, dtype: float64>

In [224]:
data["accommodates"] = data["accommodates"].astype(float)
data = data[data["accommodates"] <=  6]
data = data[data["accommodates"] >= 2]

In [325]:
# basic variables inc neighbourhood
basic_vars = [
    "accommodates",
    "beds",
    "property_type",
    "room_type",
    "neighbourhood_cleansed",
    #"host_acceptance_rate",
    #'amenities'
]

# reviews
reviews = [
    "number_of_reviews",
    "review_scores_rating",
]

In [326]:
data['host_acceptance_rate'] = data['host_acceptance_rate'].replace('%', '', regex=True).astype(float)
data['host_acceptance_rate'].info

<bound method Series.info of 0        100.0
1        100.0
2        100.0
3         99.0
4        100.0
         ...  
13596    100.0
13597     93.0
13598     93.0
13599    100.0
13600    100.0
Name: host_acceptance_rate, Length: 12197, dtype: float64>

In [327]:
predictors_1 = basic_vars
predictors_2 = basic_vars + reviews
all = ['price'] + predictors_2

In [328]:
data_selected = data[all]

In [329]:
# 少なくとも一つのカラムでNullを含む行を全て削除する
# df_cleaned = df.dropna(subset=['column_name'])
data_selected = data_selected.dropna()

In [330]:
data_selected.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10324 entries, 0 to 13524
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   price                   10324 non-null  object 
 1   accommodates            10324 non-null  float64
 2   beds                    10324 non-null  float64
 3   property_type           10324 non-null  object 
 4   room_type               10324 non-null  object 
 5   neighbourhood_cleansed  10324 non-null  object 
 6   number_of_reviews       10324 non-null  int64  
 7   review_scores_rating    10324 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 725.9+ KB


In [331]:
data_selected['price'].info

<bound method Series.info of 0        $63.00
3        $33.00
4        $64.00
5        $33.00
6        $68.00
          ...  
13503    $70.00
13507    $55.00
13519    $80.00
13523    $77.00
13524    $77.00
Name: price, Length: 10324, dtype: object>

In [332]:
# 例えば、'$1,000'という形式の価格を1000.0のようなfloatに変換する
data_selected['price'] = data_selected['price'].replace('[\$,]', '', regex=True).astype(float)

In [333]:
data_selected['price'].info

<bound method Series.info of 0        63.0
3        33.0
4        64.0
5        33.0
6        68.0
         ... 
13503    70.0
13507    55.0
13519    80.0
13523    77.0
13524    77.0
Name: price, Length: 10324, dtype: float64>

# Random Forest

In [334]:
data_train, data_holdout = train_test_split(data_selected, train_size=0.7, random_state=42)

In [335]:
data_train.shape, data_holdout.shape

((7226, 8), (3098, 8))

In [336]:
from patsy import dmatrices
y, X = dmatrices("price ~ " + " + ".join(predictors_2), data_train)

In [337]:
type(X)

patsy.design_info.DesignMatrix

In [338]:
X

DesignMatrix with shape (7226, 198)
  Columns:
    ['Intercept',
     'property_type[T.Camper/RV]',
     'property_type[T.Campsite]',
     'property_type[T.Casa particular]',
     'property_type[T.Earthen home]',
     'property_type[T.Entire bed and breakfast]',
     'property_type[T.Entire bungalow]',
     'property_type[T.Entire cabin]',
     'property_type[T.Entire chalet]',
     'property_type[T.Entire condo]',
     'property_type[T.Entire cottage]',
     'property_type[T.Entire guest suite]',
     'property_type[T.Entire guesthouse]',
     'property_type[T.Entire home]',
     'property_type[T.Entire home/apt]',
     'property_type[T.Entire loft]',
     'property_type[T.Entire place]',
     'property_type[T.Entire rental unit]',
     'property_type[T.Entire serviced apartment]',
     'property_type[T.Entire townhouse]',
     'property_type[T.Entire vacation home]',
     'property_type[T.Entire villa]',
     'property_type[T.Farm stay]',
     'property_type[T.Houseboat]',
     'prop

In [339]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state = 20240211)

In [340]:
tune_grid = {"max_features": [6, 8, 10, 12], "min_samples_leaf": [5, 10, 15]}

In [341]:
rf_random = GridSearchCV(
    rfr,
    tune_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=3,
)

In [342]:
print(X.shape)  # X は特徴量のデータセット
print(y.shape)  # y はターゲットのデータセット

(7226, 198)
(7226, 1)


In [343]:
%%time
rf_model = rf_random.fit(X, y.ravel())

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END max_features=6, min_samples_leaf=5;, score=-103.531 total time=   1.3s


[CV 2/5] END max_features=6, min_samples_leaf=5;, score=-53.818 total time=   1.0s
[CV 3/5] END max_features=6, min_samples_leaf=5;, score=-55.231 total time=   1.2s
[CV 4/5] END max_features=6, min_samples_leaf=5;, score=-64.285 total time=   0.4s
[CV 5/5] END max_features=6, min_samples_leaf=5;, score=-149.338 total time=   0.4s
[CV 1/5] END max_features=6, min_samples_leaf=10;, score=-104.214 total time=   0.4s
[CV 2/5] END max_features=6, min_samples_leaf=10;, score=-55.339 total time=   0.4s
[CV 3/5] END max_features=6, min_samples_leaf=10;, score=-56.310 total time=   0.6s
[CV 4/5] END max_features=6, min_samples_leaf=10;, score=-65.146 total time=   0.7s
[CV 5/5] END max_features=6, min_samples_leaf=10;, score=-149.817 total time=   0.4s
[CV 1/5] END max_features=6, min_samples_leaf=15;, score=-104.428 total time=   0.3s
[CV 2/5] END max_features=6, min_samples_leaf=15;, score=-55.734 total time=   0.3s
[CV 3/5] END max_features=6, min_samples_leaf=15;, score=-56.719 total time=

In [344]:
rf_model.cv_results_

{'mean_fit_time': array([0.82794461, 0.44026384, 0.33106689, 0.53073082, 0.77735782,
        0.73169017, 0.70687428, 0.45900621, 0.42521319, 0.59711809,
        0.49030356, 0.40550594]),
 'std_fit_time': array([0.36021376, 0.11986776, 0.07292353, 0.0564011 , 0.41325752,
        0.30895068, 0.15968057, 0.08388125, 0.07125794, 0.06271966,
        0.04318301, 0.00521035]),
 'mean_score_time': array([0.05233684, 0.03055301, 0.01419787, 0.01950779, 0.03843508,
        0.03040981, 0.02741232, 0.02799778, 0.01534243, 0.02012882,
        0.01687803, 0.01650472]),
 'std_score_time': array([0.05115773, 0.02229241, 0.00389355, 0.0027532 , 0.02779659,
        0.02604142, 0.0134914 , 0.02294931, 0.00316454, 0.00091676,
        0.00091879, 0.00271165]),
 'param_max_features': masked_array(data=[6, 6, 6, 8, 8, 8, 10, 10, 10, 12, 12, 12],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
        fill_value='?',
             dty

In [345]:
df_rf_model_cv_results = pd.DataFrame(rf_model.cv_results_)[[
    'param_max_features', 'param_min_samples_leaf', 'mean_test_score']]

In [346]:
df_rf_model_cv_results.columns = ['max features', 'min node size', 'RMSE']

In [347]:
df_rf_model_cv_results

,max features,min node size,RMSE
0,6,5,-85.240553
1,6,10,-86.165199
2,6,15,-86.452301
3,8,5,-84.321239
4,8,10,-85.377272
5,8,15,-85.776705
6,10,5,-83.760198
7,10,10,-84.966825
8,10,15,-85.458791
9,12,5,-83.388771


0	6	5	-96.703221
1	6	10	-96.783254
2	6	15	-96.805609
3	8	5	-96.435436
4	8	10	-96.675338
5	8	15	-96.700477
6	10	5	-96.207550
7	10	10	-96.613786
8	10	15	-96.656862
9	12	5	-96.172876
10	12	10	-96.656606
11	12	15	-96.706496

In [348]:
df_rf_model_cv_results.pivot(
    index = 'max features', 
    columns = 'min node size', 
    values = 'RMSE').round(2)*-1

min node size,5,10,15
max features,,,
6,85.24,86.17,86.45
8,84.32,85.38,85.78
10,83.76,84.97,85.46
12,83.39,84.41,84.91


In [349]:
rf_model.best_score_

-83.38877104009332

In [350]:
rf_model.best_params_

{'max_features': 12, 'min_samples_leaf': 5}

In [351]:
rf_model.best_estimator_

RandomForestRegressor(max_features=12, min_samples_leaf=5,
                      random_state=20240211)

In [352]:
rf_model.best_estimator_.feature_importances_

array([0.00000000e+00, 1.23941402e-04, 0.00000000e+00, 9.96647643e-06,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.40153289e-03, 7.53342454e-04, 5.68887452e-05,
       1.20798777e-03, 2.48882599e-02, 0.00000000e+00, 1.01631967e-02,
       7.62434966e-06, 1.33103428e-02, 9.08096709e-03, 6.63561936e-03,
       5.63677048e-05, 9.70892428e-03, 5.57054259e-05, 0.00000000e+00,
       9.22750788e-07, 0.00000000e+00, 4.46231474e-04, 0.00000000e+00,
       9.61179990e-05, 9.30166525e-05, 0.00000000e+00, 6.62085101e-04,
       0.00000000e+00, 0.00000000e+00, 3.87097902e-04, 1.40649938e-03,
       1.11053373e-02, 4.76501416e-04, 4.18716095e-06, 0.00000000e+00,
       9.14375911e-04, 3.60178498e-02, 2.00001238e-04, 0.00000000e+00,
       9.57241536e-04, 1.19941510e-03, 0.00000000e+00, 0.00000000e+00,
       1.56996693e-03, 0.00000000e+00, 5.06573464e-05, 1.95380467e-03,
       2.62153883e-05, 3.16562464e-04, 0.00000000e+00, 0.00000000e+00,
      

In [353]:
y_holdout, X_holdout = dmatrices("price ~ " + " + ".join(predictors_2), data_holdout)

In [354]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer

In [355]:
categorical_columns = [col for col in predictors_2 if col == "property_type" or col == "room_type" or col == "neighbourhood_cleansed"]
numerical_columns = [col for col in predictors_2 if col not in categorical_columns]

In [356]:
categorical_encoder = OneHotEncoder(handle_unknown="ignore")

preprocessing = ColumnTransformer([
    ("cat", categorical_encoder, categorical_columns),
    ("num", "passthrough", numerical_columns)])

rf_pipeline = Pipeline(
    [("preprocess", preprocessing), 
     ("regressor", rf_model.best_estimator_)] # put best model to pipeline
)

In [357]:
# predictors_2 = ['accommodates','beds','property_type','room_type','neighbourhood_cleansed','number_of_reviews','review_scores_rating']

In [358]:
%%time
rf_pipeline.fit(data_train[predictors_2],data_train.price)

CPU times: user 729 ms, sys: 31.4 ms, total: 760 ms
Wall time: 1.44 s


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['property_type', 'room_type',
                                                   'neighbourhood_cleansed']),
                                                 ('num', 'passthrough',
                                                  ['accommodates', 'beds',
                                                   'number_of_reviews',
                                                   'review_scores_rating'])])),
                ('regressor',
                 RandomForestRegressor(max_features=12, min_samples_leaf=5,
                                       random_state=20240211))])

In [359]:
data_holdout_w_prediction = data_holdout.assign(
    predicted_price=rf_pipeline.predict(data_holdout[predictors_2])
)

In [360]:
all_holdout = pd.DataFrame(
    [
        mean_squared_error(
            data_holdout_w_prediction.price,
            data_holdout_w_prediction.predicted_price,
            squared=False,
        ),
        data_holdout_w_prediction.price.mean(),
    ],
    index=["rmse", "mean_price"],
).T.assign(rmse_norm=lambda x: x.rmse / x.mean_price).round(2)
all_holdout.index = ["Total"]

all_holdout

,rmse,mean_price,rmse_norm
Total,62.09,80.09,0.78


# OLS

In [368]:
y, X = dmatrices("price ~ " + " + ".join(predictors_2), data_train)

ols_model = LinearRegression().fit(X,y)

y_test, X_test = dmatrices("price ~ " + " + ".join(predictors_2), data_holdout)

y_hat = ols_model.predict(X)

ols_rmse = mean_squared_error(y,y_hat,squared=False)
ols_rmse

90.13535869925323

# LASSO

In [369]:
from sklearn.linear_model import ElasticNet

In [370]:
lasso_model =  ElasticNet(l1_ratio = 1, fit_intercept = True)

In [371]:
lasso_model_cv = GridSearchCV(
    lasso_model,
    {"alpha":[i/100 for i in range(5, 26, 5)]},
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=3,
)

In [372]:
y, X = dmatrices("price ~ " + " + ".join(predictors_2), data_train)

In [373]:
%%time
lasso_model_cv.fit(X, y.ravel())

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END .....................alpha=0.05;, score=-102.511 total time=   0.4s
[CV 2/5] END ......................alpha=0.05;, score=-51.223 total time=   0.1s
[CV 3/5] END ......................alpha=0.05;, score=-52.496 total time=   0.1s
[CV 4/5] END ......................alpha=0.05;, score=-62.735 total time=   0.4s
[CV 5/5] END .....................alpha=0.05;, score=-147.916 total time=   0.1s
[CV 1/5] END ......................alpha=0.1;, score=-102.402 total time=   0.0s
[CV 2/5] END .......................alpha=0.1;, score=-51.501 total time=   0.0s
[CV 3/5] END .......................alpha=0.1;, score=-52.437 total time=   0.1s
[CV 4/5] END .......................alpha=0.1;, score=-62.943 total time=   0.1s
[CV 5/5] END ......................alpha=0.1;, score=-147.911 total time=   0.1s
[CV 1/5] END .....................alpha=0.15;, score=-102.427 total time=   0.1s
[CV 2/5] END ......................alpha=0.15;, s

GridSearchCV(cv=5, estimator=ElasticNet(l1_ratio=1),
             param_grid={'alpha': [0.05, 0.1, 0.15, 0.2, 0.25]},
             scoring='neg_root_mean_squared_error', verbose=3)

In [374]:
lasso_rmse = pd.DataFrame(lasso_model_cv.cv_results_).loc[
    lambda x: x.param_alpha == lasso_model_cv.best_estimator_.alpha
].mean_test_score.values[0] * -1
lasso_rmse

83.37626925962692

In [375]:
pd.DataFrame({'model': ['OLS', 'LASSO', 'random forest'],
              'CV RMSE': [ols_rmse, lasso_rmse, all_holdout.rmse[0]]})

,model,CV RMSE
0,OLS,90.135359
1,LASSO,83.376269
2,random forest,62.090000
